In [ ]:
import os
os.environ["FLAGS_use_mkldnn"] = "0"


In [ ]:
!git clone https://github.com/microsoft/OmniParser.git

In [ ]:
%ls

In [ ]:
%cd OmniParser

In [ ]:
!pip install -r requirements.txt

In [ ]:
!mkdir weights

In [ ]:
!python -m pip uninstall -y huggingface-hub -q
!python -m pip install -q "huggingface-hub==0.23.4"


In [ ]:
from getpass import getpass
from huggingface_hub import login

token = getpass()
login(token=token)


In [ ]:
import os
from huggingface_hub import hf_hub_download

repo = "microsoft/OmniParser-v2.0"
out_dir = "weights"
os.makedirs(out_dir, exist_ok=True)

files = [
    "icon_detect/train_args.yaml",
    "icon_detect/model.pt",
    "icon_detect/model.yaml",
    "icon_caption/config.json",
    "icon_caption/generation_config.json",
    "icon_caption/model.safetensors",
]

for f in files:
    path = hf_hub_download(
        repo_id=repo,
        filename=f,
        local_dir=out_dir,
        local_dir_use_symlinks=False,
        force_download=True,      
        resume_download=False     
)
    
    print("baixado:", path)


!mv OmniParser/weights/icon_caption OmniParser/weights/icon_caption_florence

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
files = api.list_repo_files("microsoft/OmniParser-v2.0")

for file in files:
    print(file)

In [ ]:
!ls OmniParser/weights/
!ls OmniParser/weights/icon_caption_florence

In [ ]:
import sys
sys.path.append("/content/OmniParser")

from ultralytics import YOLO

model_path = "OmniParser/weights/icon_detect/model.pt"
device = "cuda"

som_model = YOLO(model_path).to(device)


In [ ]:
!python -m pip install -q -U "transformers==4.46.3" "tokenizers==0.20.3" "accelerate==0.34.2"

In [ ]:
# two choices for caption model: fine-tuned blip2 or florence2
import importlib
# import util.utils
# importlib.reload(utils)
from util.utils import get_som_labeled_img, check_ocr_box, get_caption_model_processor, get_yolo_model
caption_model_processor = get_caption_model_processor(model_name="florence2", model_name_or_path="OmniParser/weights/icon_caption_florence/", device=device)

In [ ]:
som_model.device, type(som_model)

In [ ]:
from pathlib import Path
import sys

def find_repo_root(start: Path | None = None) -> Path:
    start = start or Path.cwd()
    for p in [start, *start.parents]:
        if (p / "requirements.txt").exists() and (p / "util" / "utils.py").exists():
            return p
    raise FileNotFoundError(

    )

REPO_ROOT = find_repo_root()
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

print("REPO_ROOT =", REPO_ROOT)

In [ ]:
import os
import os, importlib

import os
os.chdir(REPO_ROOT) 

from util.utils import get_som_labeled_img, check_ocr_box, get_caption_model_processor, get_yolo_model
from ultralytics import YOLO
from PIL import Image
device = 'cuda'
model_path='OmniParser/weights/icon_detect/model.pt'

caption_model_processor = get_caption_model_processor(model_name="florence2", model_name_or_path="OmniParser/weights/icon_caption_florence/", device=device)

som_model = get_yolo_model(model_path)

#image_path = 'imgs/google_page.png'
# image_path = 'imgs/windows_home.png'
# image_path = 'imgs/windows_multitab.png'
# image_path = 'imgs/omni3.jpg'
# image_path = 'imgs/ios.png'
image_path = 'image.png'
# image_path = 'imgs/excel2.png'

image = Image.open(image_path)
image_rgb = image.convert('RGB')
print('image size:', image.size)

box_overlay_ratio = max(image.size) / 3200
draw_bbox_config = {
    'text_scale': 0.8 * box_overlay_ratio,
    'text_thickness': max(int(2 * box_overlay_ratio), 1),
    'text_padding': max(int(3 * box_overlay_ratio), 1),
    'thickness': max(int(3 * box_overlay_ratio), 1),
}
BOX_TRESHOLD = 0.05 #mexi aqui

import time
start = time.time()
ocr_bbox_rslt, is_goal_filtered = check_ocr_box(image_path, display_img = False, output_bb_format='xyxy', goal_filtering=None, easyocr_args={'paragraph': False, 'text_threshold':0.9}, use_paddleocr=False)
text, ocr_bbox = ocr_bbox_rslt
cur_time_ocr = time.time()

dino_labled_img, label_coordinates, parsed_content_list = get_som_labeled_img(image_path, som_model, BOX_TRESHOLD = BOX_TRESHOLD, output_coord_in_ratio=True, ocr_bbox=ocr_bbox,draw_bbox_config=draw_bbox_config, caption_model_processor=caption_model_processor, ocr_text=text,use_local_semantics=True, iou_threshold=0.7, scale_img=False, batch_size=128)
cur_time_caption = time.time()

In [ ]:
# plot dino_labled_img it is in base64
import base64
import matplotlib.pyplot as plt
import io
plt.figure(figsize=(15,15))

image = Image.open(io.BytesIO(base64.b64decode(dino_labled_img)))
plt.axis('off')

plt.imshow(image)
print(len(parsed_content_list))

In [ ]:
import pandas as pd
df = pd.DataFrame(parsed_content_list)
df['ID'] = range(len(df))

display(df.drop(columns=["source"], errors="ignore"))

In [ ]:
# retornar isso num array

#parsed_content_list
parsed_with_id = []
for i, e in enumerate(parsed_content_list):
    e_clean = dict(e)
    e_clean.pop("source", None)
    e_clean["id"] = i
    parsed_with_id.append(e_clean)

parsed_with_id


# THE AGENT

In [ ]:
key = 'REDACTED_GROQ_KEY_2hBRP39kIlkeASgHtOqYWGdyb3FYprorcZHbjUeGZQAxwh2IxJha'

In [ ]:
!pip install groq

In [ ]:
import json
from groq import Groq

client = Groq(api_key=key)

SYSTEM_PROMPT = """You are a UI automation agent for Android.

Goal:
Given a task and the current screen state (a list of UI elements with IDs, text, interactivity, and bounding boxes), choose the single best NEXT action to move toward completing the task.

The screen state includes spatial information:
- Each element has a bbox = [x1, y1, x2, y2] in normalized screen coordinates (0..1).
- Use bbox to reason about layout (top bar, bottom navigation, center content), proximity, and likely UI roles.

You must output ONLY a JSON object (no markdown, no extra text).

Allowed actions:
- "tap": tap a UI element by target_id (must be interactive=true)
- "type": type text into a UI element by target_id (must be interactive=true and suitable for input)
- "swipe": swipe the screen in a direction ("up","down","left","right") when needed to reveal content
- "back": press Android back
- "wait": wait briefly if loading is likely
- "done": task completed

Spatial heuristics (use when helpful):
- Top region (small y): often status bar / search bars / page titles.
- Bottom region (large y): often navigation bar / action buttons.
- Large center elements: usually main content.
- If multiple elements match the same text intent, prefer the most plausible by position/size.

Rules:
1) Choose exactly ONE next action.
2) Prefer "tap" on an element whose content best matches the task intent (exact match > partial match > semantic match).
3) Use bbox to disambiguate: e.g., prefer a search icon near the top over a random icon in the middle.
4) If you need to type, first tap the input field, then type in the next step. (One action per step.)
5) Never invent IDs. Use only IDs provided in the screen state.
6) If no good action exists, choose "swipe" (usually "up") or "back".
7) Keep "reason" short (max 1 sentence).

Return JSON with this schema:
{
  "action": "tap|type|swipe|back|wait|done",
  "target_id": number|null,
  "text": string|null,
  "direction": "up|down|left|right"|null,
  "confidence": number,
  "reason": string
}
"""


def choose_next_action(task: str, elements: list, screen_wh=(1080, 2408)):
    payload = {
        "task": task,
        "screen": {"width": screen_wh[0], "height": screen_wh[1]},
        "elements": elements
    }

    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": json.dumps(payload, ensure_ascii=False)}
        ],
        temperature=0.0
    )

    raw = completion.choices[0].message.content.strip()

    # Tenta parsear o JSON
    try:
        action = json.loads(raw)
    except json.JSONDecodeError:
        # fallback: tenta extrair o primeiro {...}
        start = raw.find("{")
        end = raw.rfind("}")
        action = json.loads(raw[start:end+1])

    return action

task = "Make a call to the first contact of the list"
print(parsed_content_list)

action = choose_next_action(task, state["elements"], screen_wh=(1080, 2408))
print(action)


In [ ]:
import io, json, base64
from PIL import Image
from groq import Groq

GROQ_MODEL = "meta-llama/llama-4-maverick-17b-128e-instruct"

# Prompt mínimo (sem system prompt)
def build_prompt(task: str) -> str:
    return f"""
You are an Android UI automation agent. You will receive ONLY a screenshot.

Task: {task}

Decide the single best NEXT action to move toward completing the task.
Output ONLY a JSON object (no markdown, no extra text).

Allowed actions:
- "tap": tap somewhere on the screen
- "type": type text (assume keyboard is ready OR you tapped an input previously)
- "swipe": swipe the screen ("up","down","left","right")
- "back": press Android back
- "wait": wait briefly if loading is likely
- "done": task completed

Rules:
1) Choose exactly ONE next action.
2) If you tap, provide either a tap_bbox in normalized coordinates (0..1).
3) If you type, fill the "text" field.
4) Keep "reason" short (max 1 sentence).

Return JSON with this schema:
{{
  "action": "tap|type|swipe|back|wait|done",
  "text": string|null,
  "direction": "up|down|left|right"|null,
  "confidence": number,
  "reason": string,
  "tap_bbox": [number, number, number, number]|null
}}
""".strip()

def pil_to_data_url(image: Image.Image) -> str:
    buf = io.BytesIO()
    image.save(buf, format="PNG")
    b64 = base64.b64encode(buf.getvalue()).decode("utf-8")
    return f"data:image/png;base64,{b64}"

def choose_next_action_image_only(task: str, image: Image.Image, max_new_tokens=300):
    prompt = build_prompt(task)
    data_url = pil_to_data_url(image.convert("RGB"))

    resp = client.chat.completions.create(
        model=GROQ_MODEL,
        messages=[{
            "role": "user",
            "content": [
                {"type": "image_url", "image_url": {"url": data_url}},
                {"type": "text", "text": prompt}
            ]
        }],
        temperature=0.0,
        max_tokens=max_new_tokens,
    )

    raw = resp.choices[0].message.content.strip()

    # parse robusto
    try:
        return json.loads(raw)
    except json.JSONDecodeError:
        s = raw.find("{")
        e = raw.rfind("}")
        return json.loads(raw[s:e+1])


img = Image.open("/content/image.png")
action = choose_next_action_image_only("Make a call to the first contact of the list", img)
print(action)


In [ ]:
from PIL import Image, ImageDraw

def draw_action_tap_bbox(
    image: Image.Image,
    action: dict,
    color=(255, 0, 0),
    width=4,
    also_draw_point=True,
):
    """
    Draws action["tap_bbox"] (normalized 0..1) on a copy of `image`.

    Expected action format (image-only version):
      action["tap_bbox"]  -> [x1, y1, x2, y2]  (normalized)
      action["tap_point"] -> [x, y]            (normalized, optional)

    Returns: PIL.Image (new image with drawing)
    """
    img = image.convert("RGB").copy()
    W, H = img.size
    draw = ImageDraw.Draw(img)

    tap_bbox = action.get("tap_bbox", None)
    if tap_bbox is not None:
        if not (isinstance(tap_bbox, (list, tuple)) and len(tap_bbox) == 4):
            raise ValueError(f"tap_bbox must be a list of 4 numbers, got: {tap_bbox}")

        x1n, y1n, x2n, y2n = tap_bbox

        # clamp
        x1n = max(0.0, min(1.0, float(x1n)))
        y1n = max(0.0, min(1.0, float(y1n)))
        x2n = max(0.0, min(1.0, float(x2n)))
        y2n = max(0.0, min(1.0, float(y2n)))

        # convert to px
        x1 = int(round(x1n * W))
        y1 = int(round(y1n * H))
        x2 = int(round(x2n * W))
        y2 = int(round(y2n * H))

        # ensure valid ordering
        left, right = sorted([x1, x2])
        top, bottom = sorted([y1, y2])

        # draw rectangle (thicker by drawing multiple times)
        for k in range(width):
            draw.rectangle([left - k, top - k, right + k, bottom + k], outline=color)

        # optionally draw center point of bbox
        if also_draw_point:
            cx = (left + right) // 2
            cy = (top + bottom) // 2
            r = max(3, width + 2)
            draw.ellipse([cx - r, cy - r, cx + r, cy + r], outline=color, width=width)

    # If no bbox, but a point exists, draw the point
    tap_point = action.get("tap_point", None)
    if tap_bbox is None and tap_point is not None and also_draw_point:
        if not (isinstance(tap_point, (list, tuple)) and len(tap_point) == 2):
            raise ValueError(f"tap_point must be a list of 2 numbers, got: {tap_point}")

        xn, yn = tap_point
        xn = max(0.0, min(1.0, float(xn)))
        yn = max(0.0, min(1.0, float(yn)))

        x = int(round(xn * W))
        y = int(round(yn * H))
        r = max(6, width + 4)
        draw.ellipse([x - r, y - r, x + r, y + r], outline=color, width=width)

    return img


out = draw_action_tap_bbox(img, action)
display(out)


In [ ]:
## outro fluxo

STATE_PROMPT = """
You are a vision model extracting an Android UI state from a screenshot.

Return ONLY valid JSON. No markdown. No extra text.

Task: Convert the screenshot into an OmniParser-like state.

Output schema:
{
  "screen": {"width": int, "height": int},
  "elements": [
    {
      "id": int,
      "type": "text|icon|button|input|image|chip|toggle|checkbox|list_item|nav|unknown",
      "text": string,
      "interactive": boolean,
      "bbox": [x1, y1, x2, y2],
    }
  ]
}

Rules:
- bbox is normalized ratios in [0,1] relative to (width,height): [x1,y1,x2,y2], top-left to bottom-right.
- Assign ids starting from 0 in reading order: top-to-bottom, left-to-right.
- "text" must be the visible label (for icons/buttons use the best guessed label; if unknown use "").
- interactive=true only if it looks tappable/clickable (buttons, nav items, icons, input fields, toggles, list items).
- Do NOT invent hidden elements. Only what is visible.
- Include at most 60 elements. Prefer higher-salience elements (buttons, fields, nav, list items) over decorative text.
- screen width/height must match the screenshot pixel size.
"""


In [ ]:
import io, base64, json
from PIL import Image
from groq import Groq


def vlm_extract_state(image: Image.Image, model="meta-llama/llama-4-maverick-17b-128e-instruct", max_tokens=1800):
    buf = io.BytesIO()
    image.save(buf, format="PNG")
    b64 = base64.b64encode(buf.getvalue()).decode()

    resp = client.chat.completions.create(
        model=model,
        messages=[{
            "role": "user",
            "content": [
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{b64}"}},
                {"type": "text", "text": STATE_PROMPT}
            ],
        }],
        temperature=0.0,
        max_tokens=max_tokens,
    )

    raw = resp.choices[0].message.content.strip()

    # parse robusto
    try:
        return json.loads(raw)
    except json.JSONDecodeError:
        s = raw.find("{")
        e = raw.rfind("}")
        return json.loads(raw[s:e+1])


In [ ]:
img = Image.open("/content/image.png").convert("RGB")
state = vlm_extract_state(img)
print(state["elements"])


In [ ]:
import json
from PIL import Image

def ensure_state_list(state):
    # se veio como string JSON, converte
    if isinstance(state, str):
        state = json.loads(state)

    # alguns casos: dict com chave tipo {"elements":[...]}
    if isinstance(state, dict):
        if "elements" in state:
            state = state["elements"]
        elif "state" in state:
            state = state["state"]
        else:
            # tenta pegar primeiro valor que seja lista
            for v in state.values():
                if isinstance(v, list):
                    state = v
                    break

    if not isinstance(state, list):
        raise TypeError(f"state precisa ser list, veio: {type(state)}")

    return state

def draw_all_state_bboxes(img: Image.Image, state, only_interactive=False):
    state = ensure_state_list(state)

    out = img.convert("RGB").copy()
    for e in state:
        if not isinstance(e, dict):
            continue  # ignora strings etc

        bbox = e.get("bbox", None)
        if bbox is None:
            continue

        if only_interactive and not e.get("interactive", False):
            continue

        action = {"tap_bbox": bbox}
        out = draw_action_tap_bbox(out, action, color=(255, 0, 0), width=3, also_draw_point=False)

    return out

# uso:
# img = Image.open("...")  # ou sua img já carregada
out = draw_all_state_bboxes(img, state, only_interactive=False)
display(out)
